In [ ]:
scans = !ls /kaggle/input/rsna-str-pulmonary-embolism-detection/train/*/* -d
len(scans)

# We're given *7279* scans


In [ ]:
from tqdm import tqdm
count_min = 10e6
count_max = 0
for s in tqdm(scans):
    cnt, = !ls {s}/* | wc -l # this is super-slow.. probably avoiding "!" would be a good idea
    count_min = min(count_min, int(cnt))
    count_max = max(count_max, int(cnt))
count_min, count_max

# Each scan consists of number of slices, between *63* and *1083*

In [ ]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import matplotlib.animation as animation

from matplotlib import animation, rc
from pydicom import dcmread


rc('animation', html='jshtml')


def read_scan(path):
    fs = !ls -d {path}/*
    
    slices = []
    for f in fs:
        ds = dcmread(f)
        data = ds.pixel_array
        num = int(ds.InstanceNumber)
        slices.append((num, data))
    
    slices.sort()
    slices = [s[1] for s in slices]
    return slices

def create_animation(ims):
    ims = ims
    fps = 30
    nSeconds = 5

    fig = plt.figure( figsize=(9,9) )

    a = ims[0]
    im = plt.imshow(a)

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    anim = animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)
    
    return anim

s1 = scans[1]
ims = read_scan(s1)
anim = create_animation(ims)

In [ ]:
anim


# Also each slice contains bunch of metadata

In [ ]:
f,*_ = !ls -d {scans[1]}/*
meta = dcmread(f)
meta

# This is part multi-class and part multi-label competition, since some labels are mutually exclusive

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv')

cols = "pe_present_on_image	negative_exam_for_pe	qa_motion	qa_contrast	flow_artifact	rv_lv_ratio_gte_1	rv_lv_ratio_lt_1	leftsided_pe	chronic_pe	true_filling_defect_not_pe	rightsided_pe	acute_and_chronic_pe	central_pe	indeterminate".split()



fig = plt.figure( figsize=(12,12) )
pie = df[cols].sum()
plt.pie(pie, labels=pie.index)
plt.show()

# So, large number of examples appears to be negative

### *wip*